# API - IEEE

# Imports

In [1]:
#!pip install xplore

In [26]:
import pandas as pd
import numpy as np
import os
import glob
import sqlite3
import yaml
import json
import requests
#import xplore

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

from functions import read_bib
from functions import load_bib
from functions import write_yaml
from functions import write_json
from functions import write_csv
from functions import read_yaml

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Params

In [27]:
config = read_yaml("../05_Config/config_api.yaml")
display("query_params: ", query_params)

url = f"http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey={query_params['apikey']}&format={query_params['format']}&max_records={query_params['max_records']}&start_record={query_params['start_record']}&sort_order={query_params['sort_order']}&sort_field=article_number&{query_params['field_to_search1']}={query_params['search_for1']}&{query_params['field_to_search2']}={query_params['search_for2']}"
url

../05_Config/config_api.yaml read successfully


'query_params: '

{'apikey': '3jk2p7za5jtd7a39gwknygvk',
 'format': 'json',
 'max_records': 1000,
 'start_record': 1,
 'sort_order': 'asc',
 'sort_field': 'article',
 'field_to_search1': 'article_title',
 'search_for1': 'big+data+quality',
 'field_to_search2': 'None',
 'search_for2': 'None'}

'http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey=3jk2p7za5jtd7a39gwknygvk&format=json&max_records=1000&start_record=1&sort_order=asc&sort_field=article_number&article_title=big+data+quality&None=None'

In [24]:
requestsIE = requests.get(url)

if requestsIEEE.status_code >= 400:
    print('Status code: ',requestsIEEE.status_code, '\nProvavelmente foi atingido o limite de consultas diario!')

else:
    dict_api_ieee = requestsIEEE.json()
    print('\ntotal_records: ' + str(dict_api_ieee['total_records']) +\
        '\ntotal_searched: ' + str(dict_api_ieee['total_searched']))

    if (int(dict_api_ieee['total_records']) // 200) > 9:
        max_pages = (9*200)+2
    else:
        max_pages = ((int(dict_api_ieee['total_records']) // 200) *200) + 2
    list_output = []
    for start_record in range(1,max_pages,200):
        print('start_record: ', start_record)
        url = f"http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey={query_params['apikey']}&format={query_params['format']}&max_records={query_params['max_records']}&start_record={start_record}&sort_order={query_params['sort_order']}&sort_field=article_number&{query_params['field_to_search1']}={query_params['search_for1']}&{query_params['field_to_search2']}={query_params['search_for2']}"
        requestsIEEE = requests.get(url)
        dict_api_ieee = requestsIEEE.json()
        df_temp = pd.DataFrame.from_dict(dict_api_ieee['articles'])
        list_output.append(df_temp)

    df_api_original_articles = pd.concat(list_output, ignore_index = True)
    print(df_api_original_articles.shape)

    #Extract only name of author when possible
    df_api_original_articles.authors = df_api_original_articles.authors.astype('O')
    for i, x in enumerate(df_api_original_articles.authors):

        if len(str(df_api_original_articles.authors[i]).split("'full_name': '", maxsplit = 1)) > 1:
            df_api_original_articles.authors[i] = str(df_api_original_articles.authors[i]).split("'full_name': '", maxsplit = 1)[-1].split("'", maxsplit = 1)[0]
        else:
            df_api_original_articles.authors[i] = np.nan

    df_api_articles = df_api_original_articles[['title', 'abstract', 'publication_year', 'authors', 'doi', 'content_type', 'issn', 'isbn']]

    print('\ndf_api_original_articles: ', df_api_original_articles.shape)
    print('df_api_articles: ', df_api_original_articles.shape)

Status code:  403 
Provavelmente foi atingido o limite de consultas diario!


# Export to Database

In [ ]:
#Exporting to SQlite database
dbfile = '/Users/amigosdadancamooca/Documents/Impacta/Python_for_Data_Engineers/03_OutputFiles/doi.db'
tabela ='api_ieee'
db = sqlite3.connect(dbfile)
sqlDataTypes={}
for c in df_api_articles.columns:
    if df_api_articles[c].dtype.kind == 'i':  
        sqlDataTypes[c]='INTEGER'
    elif df_api_articles[c].dtype.kind == 'f':
        sqlDataTypes[c]='REAL'
    else:
        sqlDataTypes[c]='TEXT'
df_api_articles.to_sql(tabela, index=False, if_exists='replace', dtype=sqlDataTypes, con=db)   
db.commit()
db.close() 

In [29]:
#Check db content

db = sqlite3.connect(dbfile)

tabela = 'api_ieee'

query_api = pd.read_sql_query(f'select * from {tabela} LIMIT 10', db)
display(pd.read_sql_query(f'select count(*) from {tabela}', db))
display(query_api)

,count(*)
0,2000


,title,abstract,publication_year,authors,doi,content_type,issn,isbn
0,Research and Application of Data Mining in Fau...,The fault characteristics of big equipments ar...,2007,Zhigang Chen,10.1109/ICMA.2007.4304167,Conferences,2152-744X,978-1-4244-0828-3
1,Data system for the monitoring of power qualit...,"During 2006, at CN Transelectrica - OMEPA Bran...",2007,Fanica Vatra,10.1109/EPQU.2007.4424094,Conferences,2150-6655,978-84-690-9441-9
2,Gas Emergence Big Data and neural network filter,The gas supervision is a safety core of the co...,2008,Li Kun,10.1109/CHICC.2008.4605537,Conferences,2161-2927,978-7-900719-70-6
3,"Big one customer, one record and a mess - [fea...",Technology has often been seen as more importa...,2004,Mat Hanrahan,10.1049/inp:20040206,Magazines,1743-694X,None
4,Relationship between Volatility of Shibor Rate...,Using a daily sample of the 1-week Shanghai in...,2010,Lifeng Dong,10.1109/IWISA.2010.5473619,Conferences,None,978-1-4244-5872-1
5,A strategy for continuously big capacity data ...,The real time online analysis system that work...,2010,Xiao-Dong Shao,10.1109/ICACC.2010.5487001,Conferences,None,978-1-4244-5847-9
6,Keynote address: How will big pictures emerge ...,Every year since the article “How Will Big Pic...,2010,Luis A. N. Amaral,10.1109/VAST.2010.5652365,Conferences,None,978-1-4244-9488-0
7,"The Big Picture, from Grids and Clouds to Crow...",Much work is underway within the broad next ge...,2010,Nik Bessis,10.1109/3PGCIC.2010.58,Conferences,None,978-0-7695-4237-9
8,Plenary talk: Big data and real time analytics,Moore's law predicted the processor and other ...,2011,G. V. N. Appa Rao,10.1109/ICRTIT.2011.5972495,Conferences,None,978-1-4577-0589-2
9,Fault parameter estimation with data assimilat...,"A procedure to estimate, in the framework of d...",2011,Hiromichi Nagao,None,Conferences,None,978-0-9824438-3-5
